In [1]:
# Interactive Development Notebook - Algoritmo GD Project
# Load real project data and keep DataFrames in memory for development

import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
import warnings

# Add project root to path so we can import from src/
project_root = Path.cwd()
if 'src' not in sys.path:
    sys.path.insert(0, str(project_root))

print("🚀 Interactive Development Environment - Algoritmo GD Project")
print("=" * 70)

# =============================================================================
# 1. IMPORT PROJECT MODULES AND CONFIGURATION
# =============================================================================

try:
    # Import project configuration and modules
    from src.config import CONFIG, PROJECT_NAME
    from src.models import DescansosDataModel
    from base_data_project.utils import create_components
    from base_data_project.log_config import setup_logger
    
    print("✅ Project modules imported successfully")
    print(f"📁 Project: {PROJECT_NAME}")
    print(f"🗂️  Root directory: {project_root}")
    
except ImportError as e:
    print(f"❌ Error importing project modules: {e}")
    print("Make sure you're running this notebook from the project root directory")
    raise

# Configure logging
logger = setup_logger(PROJECT_NAME, log_level=logging.INFO)

# =============================================================================
# 2. CONFIGURATION AND EXTERNAL DATA SETUP
# =============================================================================

print("\n📋 Setting up configuration and external data...")

# Use the real project configuration
use_db = False  # Set to True if you want to use database, False for CSV
external_call_data = CONFIG.get('external_call_data', {
    'current_process_id': 249652,
    'api_proc_id': 999,
    'wfm_proc_id': 249652,
    'wfm_user': 'WFM',
    'start_date': '2025-01-01',
    'end_date': '2025-12-31',
    'wfm_proc_colab': None,
})

print(f"📊 Data source: {'Database' if use_db else 'CSV files'}")
print(f"📅 Date range: {external_call_data['start_date']} to {external_call_data['end_date']}")
print(f"🔢 Process ID: {external_call_data['current_process_id']}")

# =============================================================================
# 3. INITIALIZE DATA MANAGER AND COMPONENTS
# =============================================================================

print("\n🔧 Initializing data manager and components...")

try:
    # Create data manager using the project's utility function
    data_manager, process_manager = create_components(
        use_db=use_db, 
        no_tracking=True,  # Disable tracking for development
        config=CONFIG
    )
    print("✅ Data manager created successfully")
    
except Exception as e:
    print(f"❌ Error creating data manager: {e}")
    raise

# =============================================================================
# 4. LOAD PROJECT DATA INTO MEMORY
# =============================================================================

print("\n📊 Loading project data into memory...")

# Initialize the data model with real project structure
data_model = DescansosDataModel(
    project_name=PROJECT_NAME, 
    external_data=external_call_data
)

print("✅ Data model initialized")

# Context manager for data manager connection
with data_manager:
    
    # =============================================================================
    # 4.1 LOAD PROCESS DATA (Stage 1)
    # =============================================================================
    
    print("\n🔄 Stage 1: Loading process data...")
    
    try:
        # Get entities to load from configuration
        entities_dict = CONFIG.get('available_entities_processing', {})
        
        success = data_model.load_process_data(data_manager, entities_dict)
        
        if success:
            print("✅ Process data loaded successfully")
            print(f"   📋 Valid employees: {len(data_model.auxiliary_data.get('valid_emp', []))} records")
            print(f"   🏢 Unit ID: {data_model.auxiliary_data.get('unit_id')}")
            print(f"   🏭 Section ID: {data_model.auxiliary_data.get('secao_id')}")
            print(f"   👤 Position IDs: {data_model.auxiliary_data.get('posto_id_list')}")
        else:
            print("❌ Failed to load process data")
            
    except Exception as e:
        print(f"❌ Error in Stage 1: {e}")
        logger.error(f"Stage 1 error: {e}", exc_info=True)
    
    # =============================================================================
    # 4.2 LOAD DETAILED DATA FOR EACH POSITION (Stage 2)
    # =============================================================================
    
    print("\n🔄 Stage 2: Loading detailed data for positions...")
    
    posto_id_list = data_model.auxiliary_data.get('posto_id_list', [])
    
    if posto_id_list:
        # Process first position as example (you can modify this)
        posto_id = posto_id_list[0]
        print(f"📍 Processing position ID: {posto_id}")
        
        try:
            # Load colaborador info
            success = data_model.load_colaborador_info(data_manager, posto_id)
            if success:
                print(f"   ✅ Colaborador info loaded")
                df_colaborador = data_model.raw_data.get('df_colaborador')
                if df_colaborador is not None:
                    print(f"      📊 {len(df_colaborador)} employee records")
            
            # Load estimativas info  
            success = data_model.load_estimativas_info(
                data_manager, 
                posto_id, 
                external_call_data['start_date'], 
                external_call_data['end_date']
            )
            if success:
                print(f"   ✅ Estimativas info loaded")
                df_estimativas = data_model.raw_data.get('df_estimativas')
                if df_estimativas is not None:
                    print(f"      📈 {len(df_estimativas)} estimate records")
            
            # Load calendario info
            success = data_model.load_calendario_info(
                data_manager,
                external_call_data['current_process_id'],
                posto_id,
                external_call_data['start_date'],
                external_call_data['end_date']
            )
            if success:
                print(f"   ✅ Calendario info loaded")
                df_calendario = data_model.raw_data.get('df_calendario')
                if df_calendario is not None:
                    print(f"      📅 Calendar matrix: {df_calendario.shape}")
            
        except Exception as e:
            print(f"   ❌ Error loading data for position {posto_id}: {e}")
            logger.error(f"Position {posto_id} error: {e}", exc_info=True)
    
    # =============================================================================
    # 4.3 PERFORM DATA TRANSFORMATIONS (Stage 3)
    # =============================================================================
    
    print("\n🔄 Stage 3: Performing data transformations...")
    
    try:
        # Load estimativas transformations
        success = data_model.load_estimativas_transformations()
        if success:
            print("   ✅ Estimativas transformations completed")
        
        # Load colaborador transformations  
        success = data_model.load_colaborador_transformations()
        if success:
            print("   ✅ Colaborador transformations completed")
        
        # Load calendario transformations
        success = data_model.load_calendario_transformations()
        if success:
            print("   ✅ Calendario transformations completed")
        
        # Perform func_inicializa
        success = data_model.func_inicializa(
            start_date=external_call_data['start_date'],
            end_date=external_call_data['end_date'],
            fer=data_model.auxiliary_data.get('df_festivos'),
            closed_days=data_model.auxiliary_data.get('df_closed_days')
        )
        if success:
            print("   ✅ func_inicializa completed")
            
    except Exception as e:
        print(f"   ❌ Error in transformations: {e}")
        logger.error(f"Transformation error: {e}", exc_info=True)

print("\n🎉 Data loading completed!")

# =============================================================================
# 5. ORGANIZE DATAFRAMES FOR EASY ACCESS
# =============================================================================

print("\n📊 Organizing DataFrames for interactive access...")

# Extract all DataFrames from the data model
auxiliary_dataframes = {}
raw_dataframes = {}
medium_dataframes = {}
rare_dataframes = {}
formatted_dataframes = {}

# Auxiliary data
for key, value in data_model.auxiliary_data.items():
    if isinstance(value, pd.DataFrame):
        auxiliary_dataframes[key] = value

# Raw data  
for key, value in data_model.raw_data.items():
    if isinstance(value, pd.DataFrame):
        raw_dataframes[key] = value

# Medium data (transformed)
for key, value in data_model.medium_data.items():
    if isinstance(value, pd.DataFrame):
        medium_dataframes[key] = value

# Rare data (algorithm results)
for key, value in data_model.rare_data.items():
    if isinstance(value, pd.DataFrame):
        rare_dataframes[key] = value

# Formatted data (final output)
for key, value in data_model.formatted_data.items():
    if isinstance(value, pd.DataFrame):
        formatted_dataframes[key] = value

# =============================================================================
# 6. DISPLAY AVAILABLE DATAFRAMES
# =============================================================================

print("\n📋 AVAILABLE DATAFRAMES")
print("=" * 70)

all_dataframes = {
    "🗂️ AUXILIARY": auxiliary_dataframes,
    "📁 RAW": raw_dataframes, 
    "⚙️ MEDIUM (Transformed)": medium_dataframes,
    "💎 RARE (Algorithm Results)": rare_dataframes,
    "📊 FORMATTED (Final)": formatted_dataframes
}

for category, dataframes in all_dataframes.items():
    if dataframes:
        print(f"\n{category}:")
        for name, df in dataframes.items():
            print(f"   📋 {name:<25} → {df.shape[0]:>6} rows × {df.shape[1]:>3} columns")
    else:
        print(f"\n{category}: (no DataFrames yet)")

# =============================================================================
# 7. QUICK ACCESS VARIABLES AND UTILITY FUNCTIONS
# =============================================================================

print(f"\n🔗 QUICK ACCESS VARIABLES")
print("=" * 70)

# Make key DataFrames easily accessible with simple variable names
try:
    if 'valid_emp' in auxiliary_dataframes:
        valid_emp = auxiliary_dataframes['valid_emp']
        print(f"✅ valid_emp           → {valid_emp.shape}")
    
    if 'df_colaborador' in raw_dataframes:
        df_colaborador = raw_dataframes['df_colaborador']
        print(f"✅ df_colaborador      → {df_colaborador.shape}")
    
    if 'df_estimativas' in raw_dataframes:
        df_estimativas = raw_dataframes['df_estimativas']
        print(f"✅ df_estimativas      → {df_estimativas.shape}")
    
    if 'df_calendario' in raw_dataframes:
        df_calendario = raw_dataframes['df_calendario']
        print(f"✅ df_calendario       → {df_calendario.shape}")
    
    if 'matrizA_bk' in medium_dataframes:
        matrizA_bk = medium_dataframes['matrizA_bk']
        print(f"✅ matrizA_bk          → {matrizA_bk.shape}")
    
    if 'matriz2_bk' in medium_dataframes:
        matriz2_bk = medium_dataframes['matriz2_bk']
        print(f"✅ matriz2_bk          → {matriz2_bk.shape}")
    
    if 'matrizB_bk' in medium_dataframes:
        matrizB_bk = medium_dataframes['matrizB_bk']
        print(f"✅ matrizB_bk          → {matrizB_bk.shape}")
        
except Exception as e:
    print(f"⚠️ Some DataFrames may not be available yet: {e}")

# =============================================================================
# 8. UTILITY FUNCTIONS FOR DATA EXPLORATION
# =============================================================================

def explore_df(df, name="DataFrame"):
    """Explore a DataFrame with detailed information"""
    print(f"\n🔍 EXPLORING: {name}")
    print("=" * 60)
    print(f"📏 Shape: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024:.1f} KB")
    
    print(f"\n📋 Columns ({len(df.columns)}):")
    for i, col in enumerate(df.columns):
        dtype = df[col].dtype
        null_count = df[col].isnull().sum()
        print(f"   {i+1:2d}. {col:<20} ({dtype}) - {null_count} nulls")
    
    print(f"\n📊 First 3 rows:")
    print(df.head(3).to_string())
    
    # Numeric summary
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\n📈 Numeric columns summary:")
        print(df[numeric_cols].describe())
    
    return df

def compare_dfs(*dataframes, names=None):
    """Compare multiple DataFrames"""
    if names is None:
        names = [f"DataFrame_{i+1}" for i in range(len(dataframes))]
    
    print(f"\n🔄 COMPARING DATAFRAMES")
    print("=" * 60)
    
    for name, df in zip(names, dataframes):
        print(f"📋 {name:<20} → {df.shape[0]:>6} rows × {df.shape[1]:>3} columns")
    
    # Check for common columns
    if len(dataframes) > 1:
        all_columns = [set(df.columns) for df in dataframes]
        common_cols = set.intersection(*all_columns)
        
        print(f"\n🔗 Common columns ({len(common_cols)}):")
        for col in sorted(common_cols):
            print(f"   • {col}")

def show_sample_data(df_dict, category_name, n_rows=3):
    """Show sample data from DataFrames in a category"""
    print(f"\n📖 SAMPLE DATA: {category_name}")
    print("=" * 60)
    
    for name, df in df_dict.items():
        print(f"\n🔹 {name} (showing {min(n_rows, len(df))} rows):")
        if len(df) > 0:
            print(df.head(n_rows).to_string())
        else:
            print("   (empty DataFrame)")

def search_columns(pattern, df_dict=None):
    """Search for columns matching a pattern across all DataFrames"""
    if df_dict is None:
        df_dict = {**auxiliary_dataframes, **raw_dataframes, **medium_dataframes}
    
    print(f"\n🔍 SEARCHING COLUMNS: '{pattern}'")
    print("=" * 60)
    
    found = False
    for df_name, df in df_dict.items():
        matching_cols = [col for col in df.columns if pattern.lower() in col.lower()]
        if matching_cols:
            found = True
            print(f"\n📋 {df_name}:")
            for col in matching_cols:
                print(f"   • {col}")
    
    if not found:
        print(f"❌ No columns found matching '{pattern}'")

def df_info():
    """Show information about all available DataFrames"""
    print(f"\n📊 ALL DATAFRAMES INFO")
    print("=" * 70)
    
    categories = [
        ("🗂️ AUXILIARY", auxiliary_dataframes),
        ("📁 RAW", raw_dataframes),
        ("⚙️ MEDIUM", medium_dataframes),
        ("💎 RARE", rare_dataframes),
        ("📊 FORMATTED", formatted_dataframes)
    ]
    
    for category_name, df_dict in categories:
        if df_dict:
            print(f"\n{category_name}:")
            for name, df in df_dict.items():
                memory_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
                print(f"   📋 {name:<25} → {df.shape[0]:>6} rows × {df.shape[1]:>3} cols ({memory_mb:.1f} MB)")

# =============================================================================
# 9. INSTRUCTIONS AND EXAMPLES
# =============================================================================

print(f"\n🛠️ UTILITY FUNCTIONS AVAILABLE:")
print("=" * 70)
print("🔍 explore_df(dataframe, 'name')              → Detailed DataFrame exploration")
print("🔄 compare_dfs(df1, df2, names=['A', 'B'])    → Compare multiple DataFrames")  
print("📖 show_sample_data(df_dict, 'category', 5)   → Show sample data from category")
print("🔍 search_columns('pattern')                  → Find columns matching pattern")
print("📊 df_info()                                  → Show all DataFrames info")

print(f"\n💡 EXAMPLE USAGE:")
print("=" * 70)
print("# Explore specific DataFrames")
print("explore_df(valid_emp, 'Valid Employees')")
print("explore_df(df_colaborador, 'Employee Details')")
print("")
print("# Compare DataFrames")
print("compare_dfs(df_colaborador, matrizA_bk, names=['Raw', 'Processed'])")
print("")
print("# Show sample data")
print("show_sample_data(raw_dataframes, 'Raw Data', 3)")
print("")
print("# Search for specific columns")
print("search_columns('matricula')")
print("search_columns('data')")
print("")
print("# Access DataFrames directly")
print("valid_emp.head()")
print("df_colaborador.describe()")
print("matrizA_bk.columns")

print(f"\n🎯 DIRECT ACCESS TO PROJECT DATA:")
print("=" * 70)
print("📊 data_model.auxiliary_data    → Dictionary with auxiliary data")
print("📁 data_model.raw_data          → Dictionary with raw DataFrames")  
print("⚙️ data_model.medium_data       → Dictionary with transformed DataFrames")
print("💎 data_model.rare_data         → Dictionary with algorithm results")
print("📋 data_model.formatted_data    → Dictionary with final formatted data")
print("")
print("📊 auxiliary_dataframes         → Easy access to auxiliary DataFrames")
print("📁 raw_dataframes              → Easy access to raw DataFrames")
print("⚙️ medium_dataframes           → Easy access to medium DataFrames")

print(f"\n✨ READY FOR INTERACTIVE DEVELOPMENT!")
print("🔧 All project DataFrames are loaded and available in memory")
print("📝 Use the utility functions above to explore and analyze the data")
print("🚀 Start developing your data transformations!")

🚀 Interactive Development Environment - Algoritmo GD Project
2025-06-12 13:10:14,475 |     INFO | Logger initialized for algoritmo_GD
✅ Project modules imported successfully
📁 Project: algoritmo_GD
🗂️  Root directory: c:\ALCAMPO\python-algorithms\algortimo-gd
2025-06-12 13:10:14,866 |     INFO | Logger initialized for algoritmo_GD

📋 Setting up configuration and external data...
📊 Data source: CSV files
📅 Date range: 2025-01-01 to 2025-12-31
🔢 Process ID: 249652

🔧 Initializing data manager and components...
2025-06-12 13:10:14,868 |     INFO | Logger initialized for base_data_project
2025-06-12 13:10:14,869 |     INFO | Data manager for 'db' not registered, trying built-in managers
2025-06-12 13:10:14,870 |     INFO | Initialized BaseDataManager
✅ Data manager created successfully

📊 Loading project data into memory...
2025-06-12 13:10:14,871 |     INFO | Initialized DescansosDataModel
2025-06-12 13:10:14,874 |     INFO | DataContainer initialized
✅ Data model initialized
2025-06-12 1

c:\ALCAMPO\python-algorithms\algortimo-gd\src\models.py:864: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  output_final = output_final.fillna(0)
c:\ALCAMPO\python-algorithms\algortimo-gd\src\models.py:1062: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matriz_ma[non_date_columns] = matriz_ma[non_date_columns].fillna(0)


2025-06-12 13:11:14,828 |     INFO | load_m2_bd completed successfully. Created calendar matrix with shape: (4, 731)
2025-06-12 13:11:14,829 |     INFO | First few rows of created matrix:
2025-06-12 13:11:14,836 |     INFO |           0           1           2           3           4           5    \
0         Dia  2025-01-01  2025-01-01  2025-01-02  2025-01-02  2025-01-03   
1    TIPO_DIA           F           F           F           F           -   
2       TURNO           M           T           M           T           M   
3  0005039619           M           0           M           0           M   

          6           7           8           9    ...         721  \
0  2025-01-03  2025-01-04  2025-01-04  2025-01-05  ...  2025-12-27   
1           -           -           -           F  ...           -   
2           T           M           T           M  ...           M   
3           0           M           0           M  ...           0   

          722         723         724 

c:\ALCAMPO\python-algorithms\algortimo-gd\src\models.py:1547: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  matrizB_ini.loc[matrizB_ini['data'].isin(special_dates), 'min_turno'] = matrizB_ini['max_turno']
c:\ALCAMPO\python-algorithms\algortimo-gd\src\models.py:1548: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  mask_friday = (matrizB_ini['data'].isin(friday_dates)) & (matrizB_ini['turno'] == 'M')


2025-06-12 13:11:15,089 |     INFO | Columns in matrizA_og after processing: ['fk_colaborador', 'unidade', 'secao', 'posto', 'convenio', 'nome', 'matricula', 'min_dia_trab', 'max_dia_trab', 'tipo_turno', 'seq_turno', 't_total', 'l_total', 'dyf_max_t', 'q', 'c2d', 'c3d', 'cxx', 'semana_1', 'out', 'ciclo', 'data_admissao', 'data_demissao', 'fk_tipo_posto', 'h_tm_in', 'h_tm_out', 'h_tt_in', 'h_tt_out', 'h_seg_in', 'h_seg_out', 'h_ter_in', 'h_ter_out', 'h_qua_in', 'h_qua_out', 'h_qui_in', 'h_qui_out', 'h_sex_in', 'h_sex_out', 'h_sab_in', 'h_sab_out', 'h_dom_in', 'h_dom_out', 'h_fer_in', 'h_fer_out', 'limite_superior_manha', 'limite_inferior_tarde', 'emp', 'lq', 'min', 'max', 'tipo_contrato', 'ld', 'l_dom', 'lq_og', 'total_dom_fes', 'total_fes', 'total_holidays', 'descansos_atrb', 'COLABORADOR', 'LD_at', 'LQ_at', 'LRES_at', 'CXX_at', 'C2D_at', 'C3D_at']


c:\ALCAMPO\python-algorithms\algortimo-gd\src\models.py:1953: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matrizA_og = matrizA_og.merge(count_ldt, left_on='matricula', right_on='COLABORADOR', how='left').fillna(0)


2025-06-12 13:11:16,313 |     INFO | func_inicializa MatrizB creation completed successfully
2025-06-12 13:11:16,315 |     INFO | func_inicializa completed successfully
   ✅ func_inicializa completed
2025-06-12 13:11:16,421 |     INFO | Closed database session
2025-06-12 13:11:16,533 |     INFO | Disposed database engine

🎉 Data loading completed!

📊 Organizing DataFrames for interactive access...

📋 AVAILABLE DATAFRAMES

🗂️ AUXILIARY:
   📋 messages_df               →      0 rows ×   0 columns
   📋 params_lq                 →      5 rows ×   2 columns
   📋 valid_emp                 →      1 rows ×   8 columns
   📋 colabs_id_list            →      1 rows ×   0 columns
   📋 df_festivos               →     14 rows ×   2 columns
   📋 df_turnos                 →      2 rows ×   6 columns
   📋 df_calendario_passado     →      0 rows ×   0 columns
   📋 df_count                  →      0 rows ×   0 columns
   📋 df_estrutura_wfm          →   2777 rows ×   6 columns
   📋 df_feriados             